In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2
import os

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import warnings
# filter warnings
warnings.filterwarnings('ignore')


import os
print(os.listdir("/content/drive/My Drive/Colab Notebooks/left_right/"))
print(len(os.listdir("/content/drive/My Drive/Colab Notebooks/left_right/train/right")))

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

from glob import glob
import os
import numpy as np
import pandas as np

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
img_width, img_height = 240,240
train = '/content/drive/My Drive/Colab Notebooks/left_right/train'
test = "/content/drive/My Drive/Colab Notebooks/left_right/test"
nb_train_samples = 7609
nb_validation_samples = 4000

batch_size = 48

In [ ]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
from keras.applications import MobileNet
from keras.layers import Dropout,BatchNormalization
conv_base = MobileNet(weights='imagenet', include_top=False, input_shape=(240,240,3))
from keras.optimizers import RMSprop,Adam
from keras import layers
from keras import models

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(BatchNormalization())
model.add(layers.Dense(512, activation='relu'))
model.add(Dropout(.4))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1 / 255.0,
    rotation_range=20,
    zoom_range=0.05,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest")
 
# initialize the validation (and testing) data augmentation object
valid_datagen = ImageDataGenerator(rescale=1 / 255.0)

In [ ]:
train_generator = train_datagen.flow_from_directory(train,
    target_size=(img_width, img_height),
    batch_size=batch_size,
           class_mode='binary')
validation_generator = valid_datagen.flow_from_directory(
    test,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
  )

In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(.0001), metrics=['acc'])

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

In [ ]:
model.save('/content/left_right.h5')

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
import numpy as np
model= load_model('/content/left_right.h5')

In [ ]:
img = image.load_img('/content/drive/My Drive/Colab Notebooks/left_right/validation/left/9995_left.jpg', target_size=(240,240))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
img_data = preprocess_input(x)
classes = model.predict(img_data)

if classes < .5:
  print('Left Eye')

else:
  print('Right Eye')